# 의도분류 모델 만들기 (학습)
train data는 total_train_data.csv입니다. 

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

## 데이터 전처리 
참고 : tf.data를 활용하는 법    
https://blog.naver.com/gyungsumin/221737330099

In [2]:
# 데이터 읽어오기
train_file = "total_train_data.csv"
data = pd.read_csv(train_file, delimiter=',') # delimiter는 구분자 (seperator) csv파일에서는 굳이 구분자 ,를 지정해주지 않아도 잘 불러오긴 함!
queries = data['query'].tolist()
intents = data['intent'].tolist()

In [4]:
queries[:5]

['헬로우',
 '헬로',
 '안부 인사드립니다.',
 '먼저 인사하려고 했는데 짝남이 먼저 인사해줬어. 더 떨렸겠어요.',
 '먼저 인사할까 했는데 짝녀가 먼저 인사해줬어. 기분 좋았겠네요.']

In [5]:
intents[:5]

[0, 0, 0, 0, 0]

In [7]:
data.head()

,query,intent
0,헬로우,0
1,헬로,0
2,안부 인사드립니다.,0
3,먼저 인사하려고 했는데 짝남이 먼저 인사해줬어. 더 떨렸겠어요.,0
4,먼저 인사할까 했는데 짝녀가 먼저 인사해줬어. 기분 좋았겠네요.,0


In [3]:
import sys
sys.path.append('/Users/zhenxi/Desktop/for_git/AIB_PJ1/AIB_PJ1_QnA-chatbot/chatbot')
from utils.Preprocess import Preprocess
p = Preprocess(word2index_dic='../../train_tools/dict/chatbot_dict.bin',
               userdic='../../utils/user_dic.tsv')

In [4]:
# 단어 시퀀스 생성
# queries에서 한문장문장씩 꺼내서 인덱스로 반환한다.

sequences = []
for sentence in queries:
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)

In [13]:
sequences[:5]

[[1],
 [6708],
 [2967, 1051, 66],
 [684, 1051, 2, 567, 896, 684, 1051, 8, 149, 1135],
 [684, 1051, 2, 567, 666, 684, 1051, 8, 732, 162]]

In [5]:
# 패딩 작업
# 단어 인덱스 시퀀스 벡터 생성
# 단어 시퀀스 벡터 크기 : config를 참고하면 MAX_SEQ_LEN은 15이다. 
from config.GlobalParams import MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [15]:
padded_seqs[:5]

array([[   1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [6708,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [2967, 1051,   66,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [ 684, 1051,    2,  567,  896,  684, 1051,    8,  149, 1135,    0,
           0,    0,    0,    0],
       [ 684, 1051,    2,  567,  666,  684, 1051,    8,  732,  162,    0,
           0,    0,    0,    0]], dtype=int32)

In [6]:
# 학습용, 검증용, 테스트용 데이터셋 생성
# 학습셋:검증셋:테스트셋 = 7:2:1

# tf.data.Dataset은 대량의 데이터를 표현할 수 있는 API
# from_tensor_slices에 두개의 리스트를 넣어줌 -> 같은 인덱스의 값끼리 튜플로 엮임 -> data split을 한번에 할 수 있다. 
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intents))

2021-12-20 13:33:10.103714: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# from_tensor_slices 결과 확인
list(ds)[-5:-1]

[(<tf.Tensor: shape=(15,), dtype=int32, numpy=
  array([ 162,  615,   19,  779, 1215,   19,  366,  539,   26,   25,  845,
          295,   19,   65,   98], dtype=int32)>,
  <tf.Tensor: shape=(), dtype=int32, numpy=4>),
 (<tf.Tensor: shape=(15,), dtype=int32, numpy=
  array([ 514,  649,   94, 7525,  649,    2,  293,  572,    0,    0,    0,
            0,    0,    0,    0], dtype=int32)>,
  <tf.Tensor: shape=(), dtype=int32, numpy=4>),
 (<tf.Tensor: shape=(15,), dtype=int32, numpy=
  array([11236,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0], dtype=int32)>,
  <tf.Tensor: shape=(), dtype=int32, numpy=4>),
 (<tf.Tensor: shape=(15,), dtype=int32, numpy=
  array([11237,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0], dtype=int32)>,
  <tf.Tensor: shape=(), dtype=int32, numpy=4>)]

In [8]:
ds = ds.shuffle(len(queries)) # 데이터를 랜덤으로 섞음

# dataset = dataset.shuffle(len(sequences))
 # shuffle 함수의 인자는 데이터의 전체 길이를 고정으로 넣어 줌

In [9]:
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

In [28]:
train_size, val_size, test_size

(73960, 21131, 10565)

In [10]:
train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

In [11]:
# 하이퍼파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(p.word_index) + 1  # 전체 단어 수

In [40]:
VOCAB_SIZE, MAX_SEQ_LEN

(17751, 15)

## 모델 정의

In [12]:
# CNN 모델 정의
input_layer = Input(shape=(MAX_SEQ_LEN,))

embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
# keras emddeing 인자 (총 단어 수, 임베딩해서 나올 사이즈, 패딩처리된 시퀀스 벡터의 크기)
# 임베딩은 왜 해주는 걸까?

dropout_emb = Dropout(rate=dropout_prob)(embedding_layer) # 과적합 방지

Conv1D를 이용하는 이유는 
글자를 1차원 데이터로 처리하기 때문

In [13]:
conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb) # input이 dropout_emb
pool1 = GlobalMaxPool1D()(conv1)

In [14]:
conv2 = Conv1D(
    filters=128, # 필터 개수
    kernel_size=4, # 필터 사이즈 
    padding='valid', # valid = 패딩 없음
    activation=tf.nn.relu)(dropout_emb) # 활성화 함수는 ReLU 
pool2 = GlobalMaxPool1D()(conv2) 

In [15]:
conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)


In [16]:
# 마지막 클래스 분류 작업을 위해 3개의 feature map을 묶어준다.(합친다!)
concat = concatenate([pool1, pool2, pool3])

In [52]:
concat

<KerasTensor: shape=(None, 384) dtype=float32 (created by layer 'concatenate_1')>

특징 추출이 끝났으니, 클래스 분류를 해야한다.   
Dense layer(hidden)를 추가하고, drop out도 해주며 마지막에 5개의 의도 중 하나로 분류 한다.

In [17]:
hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(5, name='logits')(dropout_hidden)
predictions = Dense(5, activation=tf.nn.softmax)(logits)

In [18]:
# 모델 생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Epoch 1/5
3698/3698 [==============================] - 71s 19ms/step - loss: 0.0461 - accuracy: 0.9860 - val_loss: 0.0128 - val_accuracy: 0.9930
Epoch 2/5
3698/3698 [==============================] - 70s 19ms/step - loss: 0.0146 - accuracy: 0.9944 - val_loss: 0.0134 - val_accuracy: 0.9951
Epoch 3/5
3698/3698 [==============================] - 76s 20ms/step - loss: 0.0124 - accuracy: 0.9956 - val_loss: 0.0076 - val_accuracy: 0.9970
Epoch 4/5
3698/3698 [==============================] - 72s 20ms/step - loss: 0.0101 - accuracy: 0.9965 - val_loss: 0.0065 - val_accuracy: 0.9970
Epoch 5/5
3698/3698 [==============================] - 71s 19ms/step - loss: 0.0082 - accuracy: 0.9966 - val_loss: 0.0069 - val_accuracy: 0.9965


In [20]:
# 모델 평가(테스트 데이터셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

529/529 [==============================] - 1s 2ms/step - loss: 0.0054 - accuracy: 0.9973
Accuracy: 99.734974
loss: 0.005362


In [21]:
# 모델 저장
model.save('intent_model.h5')